In [1]:
# importing libraries
from zipfile import ZipFile
import numpy as np
import openpyxl
import pandas as pd

## Setting read_zip funcion and colums names

In [2]:
def read_zip(year):
    # specifying the zip file directory
    file_name = f"/mnt/d/Mexico_Births/sinac_{year}.zip"
    
    # opening the zip file in READ mode
    with ZipFile(file_name, 'r') as zip:
        # listing all the contents of the zip file
        file_list = zip.filelist
        # getting a DataFrame from the file in the index 0
        data = pd.read_csv(zip.open(file_list[0].filename) )
    return data

In [3]:
# columns names for different year 
columns_names = {"columns_20" : ["EDAD","ESTADOCONYUGAL","ESCOLARIDAD","CLAVEOCUPACIONHABITUAL","FECHANACIMIENTO",\
"HORANACIMIENTO", "SEXO", "TALLA", "PESO", "CLUES", "ENTIDADFEDERATIVAPARTO", "MUNICIPIOPARTO",\
"LOCALIDADPARTO"], 

"columns_19" : ["EDADM","EDOCIVIL","NIV_ESCOL","CVEOCUPHAB","FECH_NACH",\
"HORA_NACH", "SEXOH", "TALLAH", "PESOH", "CLUES", "ENT_NAC", "MPO_NAC",\
"LOC_NAC"],

"columns_16" : ["edad_madre","estado_conyugal","escolaridad_madre","ocupacion_habitual_madre","fecha_nacimiento_nac_vivo",\
"hora_nacimiento_nac_vivo", "sexo_nac_vivo", "talla_nac_vivo", "peso_nac_vivo", "clues", "entidad_nacimiento", "municipio_nacimiento",\
"localidad_nacimiento"],

"new_columns": ["estado_conyugal_madre","escolaridad_madre","ocupacion_madre", "edad_madre","fecha_nacimiento_vivo",\
"hora_nacimiento_vivo", "sexo_vivo", "talla_vivo", "peso_vivo", "clues", "entidad_nacimiento", "municipio_nacimiento",\
"localidad_nacimiento"]
}

## Reading auxiliar tables

In [4]:
# reading births_colnames file which contains tables to join on main data
file_name = f"/mnt/d/Mexico_Births/births_colnames.xlsx"
aux_data = pd.read_excel(file_name, sheet_name= None) # using sheet_name=None to get all sheets from excel in a dict
aux_sheets = tuple(aux_data.keys())[4:] # taking sheets name to join with births data 
print(aux_sheets) 

('estado_conyugal', 'sexo', 'escolaridad', 'ocupacion2020', 'ocupacion2019')


In [5]:
aux_data[aux_sheets[0]] # estado counyugal table

,Clave_2020,Clave_2019_2016,desc_hom_cony
0,5,11,CASADA
1,1,12,SOLTERA
2,3,13,DIVORCIADA
3,2,14,VIUDA
4,4,15,UNIÓN LIBRE
5,6,16,SEPARADA
6,0,88,NO ESPECIFICADO
7,9,99,SE IGNORA


In [6]:
aux_data[aux_sheets[1]] # genero table

,Clave_2020,Clave_2019_2016,desc_hom_sex
0,0,0,NO ESPECIFICADO
1,1,1,HOMBRE
2,2,2,MUJER
3,9,9,SE IGNORA


In [7]:
aux_data[aux_sheets[2]].head() # escolaridad table

,Clave_2020,Clave_2019_2016,desc_hom_esc
0,0,0,NO ESPECIFICADO
1,1,1,NINGUNA
2,31,3,PRIMARIA COMPLETA
3,32,2,PRIMARIA INCOMPLETA
4,51,5,SECUNDARIA COMPLETA


## Processing data from 2016 to 2020

In [12]:
def get_unified_data(year):
    data = read_zip(year) # reading zip file to get a DF

    # selecting columns
    if year == 2020:
        key_ = f"Clave_{str(year)}"
        selected_columns = columns_names[f"columns_{str(year)[-2:]}"]
        job_id = 3
    elif year in [2019, 2018, 2017]:
        selected_columns = columns_names[f"columns_19"]
        key_ = f"Clave_2019_2016"
        job_id = 4
    else:
        selected_columns = columns_names[f"columns_16"]
        key_ = f"Clave_2019_2016"
        job_id = 4
     
    data = data[selected_columns] 
    
    # joining with auxiliar tables
    data = pd.merge(data, aux_data[aux_sheets[0]], how="left", left_on=selected_columns[1], right_on=key_)
    data = pd.merge(data, aux_data[aux_sheets[1]], how="left", left_on=selected_columns[6], right_on=key_)
    data = pd.merge(data, aux_data[aux_sheets[2]], how="left", left_on=selected_columns[2], right_on=key_)
    data = pd.merge(data, aux_data[aux_sheets[job_id]], how="left", left_on=selected_columns[3], right_on=key_)

    # sorting and renaming columns
    only_columns = [-9, -3, -1, 0, 4, 5, 6, 7, 8, 9, 10, 11, 12]
    data = data.iloc[:,only_columns]
    data.columns = columns_names["new_columns"]

    return data

In [13]:
data = get_unified_data(2019)

/tmp/ipykernel_22401/3537065925.py:2: DtypeWarning: Columns (7,51,56,63,65) have mixed types.Specify dtype option on import or set low_memory=False.
  data = read_zip(year) # reading zip file to get a DF
/tmp/ipykernel_22401/3537065925.py:24: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Clave_2019_2016_x'} in the result is deprecated and will raise a MergeError in a future version.
  data = pd.merge(data, aux_data[aux_sheets[job_id]], how="left", left_on=selected_columns[3], right_on=key_)


In [14]:
data.sample(3)

,estado_conyugal_madre,escolaridad_madre,ocupacion_madre,edad_madre,fecha_nacimiento_vivo,hora_nacimiento_vivo,sexo_vivo,talla_vivo,peso_vivo,clues,entidad_nacimiento,municipio_nacimiento,localidad_nacimiento
1197261,CASADA,SECUNDARIA COMPLETA,NO OCUPADO,36,22/12/2019,14:02:00,2,52,3700,NLIMS000636,19,6,1
33588,UNIÓN LIBRE,PRIMARIA COMPLETA,NO REMUNERADO - AMA DE CASA,34,05/01/2019,20:40:00,1,50,3690,BCSSA000015,2,1,1
173089,UNIÓN LIBRE,LICENCIATURA O PROFESIONAL INCOMPLETO,NO REMUNERADO - AMA DE CASA,23,17/01/2019,04:17:00,1,49,3570,CSSSA009244,7,89,1


## Processing data from 2015

In [4]:
data = read_zip(2015)

/tmp/ipykernel_24239/3592555958.py:1: DtypeWarning: Columns (5,6,7,54,60) have mixed types.Specify dtype option on import or set low_memory=False.
  data = read_zip(2015)


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2145199 entries, 0 to 2145198
Data columns (total 61 columns):
 #   Column                              Dtype 
---  ------                              ----- 
 0   edo_captura                         object
 1   edo_nac_madre                       object
 2   mpo_nac_madre                       object
 3   fecha_nac_madre                     object
 4   edad_madre                          int64 
 5   madre_se_considera_indigena         object
 6   madre_habla_lengua_indigena         object
 7   lengua_indigena_hablada             object
 8   estado_conyugal                     object
 9   entidad_residencia_madre            object
 10  municipio_residencia_madre          object
 11  localidad_residencia_madre          object
 12  numero_embarazos                    int64 
 13  hijos_nacidos_muertos               int64 
 14  hijos_nacidos_vivos                 int64 
 15  hijos_sobrevivientes                int64 
 16  el_hijo_anterior_n

In [6]:
selected_columns = columns_names[f"columns_16"]

In [7]:
selected_columns

['edad_madre',
 'estado_conyugal',
 'escolaridad_madre',
 'ocupacion_habitual_madre',
 'fecha_nacimiento_nac_vivo',
 'hora_nacimiento_nac_vivo',
 'sexo_nac_vivo',
 'talla_nac_vivo',
 'peso_nac_vivo',
 'clues',
 'entidad_nacimiento',
 'municipio_nacimiento',
 'localidad_nacimiento']

In [8]:
data = data[selected_columns]

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2145199 entries, 0 to 2145198
Data columns (total 13 columns):
 #   Column                     Dtype 
---  ------                     ----- 
 0   edad_madre                 int64 
 1   estado_conyugal            object
 2   escolaridad_madre          object
 3   ocupacion_habitual_madre   object
 4   fecha_nacimiento_nac_vivo  object
 5   hora_nacimiento_nac_vivo   object
 6   sexo_nac_vivo              object
 7   talla_nac_vivo             int64 
 8   peso_nac_vivo              int64 
 9   clues                      object
 10  entidad_nacimiento         object
 11  municipio_nacimiento       object
 12  localidad_nacimiento       object
dtypes: int64(3), object(10)
memory usage: 212.8+ MB


In [10]:
data.sample(3)

,edad_madre,estado_conyugal,escolaridad_madre,ocupacion_habitual_madre,fecha_nacimiento_nac_vivo,hora_nacimiento_nac_vivo,sexo_nac_vivo,talla_nac_vivo,peso_nac_vivo,clues,entidad_nacimiento,municipio_nacimiento,localidad_nacimiento
1115866,38,UNIÓN LIBRE,PRIMARIA COMPLETA,NO REMUNERADO - AMA DE CASA,16/08/2015,02:04,HOMBRE,50,3500,QRSSA001044,QUINTANA ROO,BENITO JUAREZ,CANCUN
918457,34,CASADA,SECUNDARIA COMPLETA,NO REMUNERADO - AMA DE CASA,11/11/2015,08:40,HOMBRE,51,4000,TSIMS000276,TAMAULIPAS,NUEVO LAREDO,NUEVO LAREDO
709336,28,CASADA,SECUNDARIA COMPLETA,NO REMUNERADO - AMA DE CASA,01/10/2015,16:27,HOMBRE,99,1800,MSIMS000054,MORELOS,CUERNAVACA,CUERNAVACA


In [11]:
data["estado_conyugal"].unique()

array(['UNIÓN LIBRE', 'SOLTERA', 'CASADA', 'S.I.', 'VIUDA', 'SEPARADA',
       'DIVORCIADA', 'N.E.'], dtype=object)

In [12]:
data["escolaridad_madre"].unique()

array(['SECUNDARIA COMPLETA', 'SECUNDARIA INCOMPLETA',
       'BACHILLERATO O PREPARATORIA INCOMPLETA', 'PRIMARIA COMPLETA',
       'PROFESIONAL', 'BACHILLERATO O PREPARATORIA COMPLETA',
       'PRIMARIA INCOMPLETA', 'NINGUNA', 'S.I.', 'POSGRADO', 'N.E.',
       'PROFESIONAL INCOMPLETO', 'POSGRADO INCOMPLETO'], dtype=object)

In [13]:
data["ocupacion_habitual_madre"].unique()

array(['NO REMUNERADO - AMA DE CASA', 'TÉCNICOS',
       'TRABAJADORES SERVICIOS DOMÉSTICOS (ASEO, CONSERJE, COCINERO, COSTURERA, PANADERO, MECÁNICO,CARGADOR)',
       'NO REMUNERADO - ESTUDIANTE', 'NO OCUPADO', 'PROFESIONISTAS',
       'COMERCIANTES, AMBULANTES, DEPENDIENTES, AGENTES DE VENTAS',
       'EMPLEADOS DE SECTORES PÚBLICO Y PRIVADO',
       'ARTESANOS, FABRILES, OPERADORES, AYUDANTES Y PEONES EN PROCESOS ARTESANALES E INDUSTRIALES,ALBAÑILES',
       'TRABAJADORES SERVICIOS PERSONALES', 'OTROS',
       'TRABAJADORES DEL ARTE, ESPECTÁCULOS Y DEPORTES',
       'TRABAJADORES SERVICIOS DE TRANSPORTE',
       'JEFES DE DEPARTAMENTO, COORDINADORES Y SUPERVISORES DE ACTIVIDADES INDUSTRIALES',
       'TRABAJADORES DE LA EDUCACIÓN', 'N.E.',
       'TRABAJADORES EN SERVICIOS DE PROTECCIÓN (SOLDADOS, FEDERALES, MARINA) Y VIGILANCIA',
       'FUNCIONARIOS Y DIRECTIVOS DE SECTORES PÚBLICO, PRIVADO Y SOCIAL',
       'S.I.',
       'TRABAJADORES EN ACTIVIDADES AGRÍCOLAS, GANADERAS, SILVÍCO

In [15]:
[('S.I.','SE IGNORA'),('N.E.','NO ESPECIFICADO')]

('S.I.', 'SE IGNORA')